In [1]:
from CustomFunctions import PILRagg
from CustomFunctions import linear_cycle_utils
from aicscytoparam import cytoparam
import re
import vtk
import multiprocessing
import numpy as np
import pandas as pd
import os
import itertools
from pathlib import Path
import glob
from aicsimageio.writers.ome_tiff_writer import OmeTiffWriter

def collect_results(result):
    """Uses apply_async's callback to setup up a separate Queue for each process.
    This will allow us to collect the results from different threads."""
    results.append(result)

In [2]:
#get directories and open separated datasets
treatment = 'random'
time_interval = 5 #sec/frame

basedir = 'E:/Aaron/random_lls_smooth/'
datadir = basedir + 'Data_and_Figs/'
savedir = basedir + 'random/'
pilr_fl = basedir + 'PILRs/'
pilragg_fl = savedir + 'avgPILRs/'
if not os.path.exists(pilragg_fl):
    os.makedirs(pilragg_fl)
    
binframe = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
binframe = binframe[['cell']+[x for x in binframe.columns.to_list() if 'bin' in x]]

TotalFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)
TotalFrame = TotalFrame.drop(columns = [x for x in TotalFrame.columns.to_list() if 'bins' in x])
TotalFrame = TotalFrame.merge(binframe, right_on = 'cell', left_on = 'cell')

centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)

nbins = np.max(TotalFrame[[x for x in TotalFrame.columns.to_list() if 'bin' in x]].to_numpy())

#restrict dataframe to only random experiments
df = TotalFrame[TotalFrame.Treatment=='Random'].copy()

In [3]:
############## Average PILRs of entire dataset and split by migration mode #########################    
#specify number of PCs and number of bins
npcs = 10
nbins = 5
bins = list(range(1,nbins+1))
nisos = [1,63]
lmax = 10

#make tiny sphere for inner mesh
#create inner sphere
sphereSource = vtk.vtkSphereSource()
sphereSource.SetCenter(0.0, 0.0, 0.0)
sphereSource.SetRadius(nisos[0]/2)
# Make the surface smooth.
sphereSource.SetPhiResolution(100)
sphereSource.SetThetaResolution(100)
sphereSource.Update()
spherepoly = sphereSource.GetOutput()





pcdf = df[[x for x in df.columns.to_list() if 'bins' in x]]
for g in df.structure.unique():
    for i, p in enumerate(pcdf.columns.to_list()):
        for b in bins:
            allpilrs = []

            temp = pcdf[p][df.structure==g]
            num = temp[temp==b].cell.to_list()
            if __name__ ==  '__main__':
                # use multiprocessing to perform segmentation and x,y,z determination
                pool = multiprocessing.Pool(processes=60)
                results= []
                for n in num:
                    pool.apply_async(PILRagg.read_parameterized_intensity, args=(pilr_fl + n + '_PILR.tiff',), 
                                     callback=collect_results)
                pool.close()
                pool.join()
            pagg = np.array(results)
            allpilrs.append(pagg)
            #get average representation
            pagg_avg = np.mean(pagg, axis = 0)
            #normalize representations
            pagg_norm = PILRagg.normalize_representations(pagg)
            pagg_norm_avg = np.mean(pagg_norm, axis = 0)
            dims = [['X', 'Y', 'Z', 'C', 'T'][d] for d in range(pagg_avg.ndim)]
            dims = ''.join(dims[::-1])
            OmeTiffWriter.save(pagg_avg, pilragg_fl+p[:-1]+str(b)+f'_{g}_repsagg.tif', dim_order=dims)
            OmeTiffWriter.save(pagg_norm_avg, pilragg_fl+p[:-1]+str(b)+f'_{g}_repsagg_norm.tif', dim_order=dims)


            mesh_outer = PILRagg.read_vtk_polydata(datadir+'PC_Meshes/'+f'Cell_PC{i+1}_{b}_Cell.vtp')
            domain, origin = cytoparam.voxelize_meshes([mesh_outer, spherepoly])
            coords_param, _ = cytoparam.parameterize_image_coordinates(
                seg_mem=(domain>0).astype(np.uint8),
                seg_nuc=(domain>1).astype(np.uint8),
                lmax=lmax,
                nisos=nisos
            )

            morphed = cytoparam.morph_representation_on_shape(
                        img=domain,
                        param_img_coords=coords_param,
                        representation=pagg_avg)
            morphed = np.stack([domain, morphed])
            OmeTiffWriter.save(morphed, pilragg_fl+p[:-1]+str(b)+f'_{g}_aggmorph.tif', dim_order='CZYX')

            print(f'Done with {g} {p} bin {b}')

Done with myosin PC1_bins bin 1
Done with myosin PC1_bins bin 2
Done with myosin PC1_bins bin 3
Done with myosin PC1_bins bin 4
Done with myosin PC1_bins bin 5
Done with myosin PC2_bins bin 1
Done with myosin PC2_bins bin 2
Done with myosin PC2_bins bin 3
Done with myosin PC2_bins bin 4
Done with myosin PC2_bins bin 5
Done with myosin PC3_bins bin 1
Done with myosin PC3_bins bin 2
Done with myosin PC3_bins bin 3
Done with myosin PC3_bins bin 4
Done with myosin PC3_bins bin 5
Done with myosin PC4_bins bin 1
Done with myosin PC4_bins bin 2
Done with myosin PC4_bins bin 3
Done with myosin PC4_bins bin 4
Done with myosin PC4_bins bin 5
Done with myosin PC5_bins bin 1
Done with myosin PC5_bins bin 2
Done with myosin PC5_bins bin 3
Done with myosin PC5_bins bin 4
Done with myosin PC5_bins bin 5
Done with myosin PC6_bins bin 1
Done with myosin PC6_bins bin 2
Done with myosin PC6_bins bin 3
Done with myosin PC6_bins bin 4
Done with myosin PC6_bins bin 5
Done with myosin PC7_bins bin 1
Done wit

In [4]:
########### linearize random untreated data and animate them

origin = [7, 6]
whichpcs = [2,7]
binrange = 10
direction = 'clockwise'
zerostart = 'left'


#### linearize cycle in CGPS
angframe = linear_cycle_utils.linearize_cycle_continuous(
            TotalFrame, 
            centers,
            origin, 
            whichpcs,
            zerostart,)

angframe =  linear_cycle_utils.bin_angular_coord(
        angframe,
        whichpcs,
        direction,
        binrange,
        )

coeff_df = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
coeffframe = angframe.merge(coeff_df[[x for x in coeff_df.columns.to_list() if 'shco' in x]+['cell']],
                              left_on = 'cell', right_on = 'cell')
for t, treat in coeffframe.groupby('Treatment'):
    linear_cycle_utils.animate_linear_cycle_shcoeffs(
                            coeffframe,
                            savedir,
                            t,
                            whichpcs,
                            binrange,
                            lmax = 10,
                            smooth = False
                            )

#### get the average PILRs around the cycle
if __name__ ==  '__main__':
    linear_cycle_utils.get_linear_cycle_PILRs(
                        angframe, #pandas dataframe with the CGPS angular coordinates
                        savedir,
                        pilr_fl,
                        )

#### combine the PILRs around the cycle into one movie
projtype = ['sum','max','mean','slice']
for t, treat in coeffframe.groupby('Treatment'):
    structurelist=[]
    for x in os.listdir(savedir + f'/PC{whichpcs[0]}-PC{whichpcs[1]}_Cycle_AllSHCoeff_Visualization/{t}/avgPILRs/'):
        struct = x.split('_')[-2]
        if struct not in structurelist and 'aggmorph' in x: structurelist.append(struct)
    for structure in structurelist:
        linear_cycle_utils.combine_linear_PILRs(savedir,
                                                t,
                                                structure,
                                                whichpcs,
                                                projtype,
                                                )

Finished frame_0_Random_actin
Finished frame_10_Random_actin
Finished frame_20_Random_actin
Finished frame_30_Random_actin
Finished frame_40_Random_actin
Finished frame_50_Random_actin
Finished frame_60_Random_actin
Finished frame_70_Random_actin
Finished frame_80_Random_actin
Finished frame_90_Random_actin
Finished frame_100_Random_actin
Finished frame_110_Random_actin
Finished frame_120_Random_actin
Finished frame_130_Random_actin
Finished frame_140_Random_actin
Finished frame_150_Random_actin
Finished frame_160_Random_actin
Finished frame_170_Random_actin
Finished frame_180_Random_actin
Finished frame_190_Random_actin
Finished frame_200_Random_actin
Finished frame_210_Random_actin
Finished frame_220_Random_actin
Finished frame_230_Random_actin
Finished frame_240_Random_actin
Finished frame_250_Random_actin
Finished frame_260_Random_actin
Finished frame_270_Random_actin
Finished frame_280_Random_actin
Finished frame_290_Random_actin
Finished frame_300_Random_actin
Finished frame_310_

In [3]:
##################### DO ALL OF THE RANDOM DATA PILR CORRELATION ##########

#get list of pilrs that actually exist and are in the random dataset
randompilrlist = [pilr_fl + x for x in os.listdir(pilr_fl) if x.split('_PILR')[0] in df.cell.to_list()]
#get all pairwise combos of said pilrs
combos = list(itertools.combinations(randompilrlist, 2))
print('Finished making combos list')
#get the correlations with multiprocessing
start = 0
stop = 1000
allresults = []
while start<len(combos):
    print(f'Finished {start}, starting {start}-{stop}')
    if __name__ == '__main__':
        pool = multiprocessing.Pool(processes=60)
        corrlist = pool.map(PILRagg.pilr_correlation, combos[start:stop])
        pool.close()
        pool.join()

    allresults.extend(corrlist)
    
    start = stop + 1
    stop = stop + 1000
    if stop>len(combos):
        stop = len(combos)

Finished making combos list
Finished 0, starting 0-1000
Finished 1001, starting 1001-2000
Finished 2001, starting 2001-3000
Finished 3001, starting 3001-4000
Finished 4001, starting 4001-5000
Finished 5001, starting 5001-6000
Finished 6001, starting 6001-7000
Finished 7001, starting 7001-8000
Finished 8001, starting 8001-9000
Finished 9001, starting 9001-10000
Finished 10001, starting 10001-11000
Finished 11001, starting 11001-12000
Finished 12001, starting 12001-13000
Finished 13001, starting 13001-14000
Finished 14001, starting 14001-15000
Finished 15001, starting 15001-16000
Finished 16001, starting 16001-17000
Finished 17001, starting 17001-18000
Finished 18001, starting 18001-19000
Finished 19001, starting 19001-20000
Finished 20001, starting 20001-21000
Finished 21001, starting 21001-22000
Finished 22001, starting 22001-23000
Finished 23001, starting 23001-24000
Finished 24001, starting 24001-25000
Finished 25001, starting 25001-26000
Finished 26001, starting 26001-27000
Finished

Finished 213001, starting 213001-214000
Finished 214001, starting 214001-215000
Finished 215001, starting 215001-216000
Finished 216001, starting 216001-217000
Finished 217001, starting 217001-218000
Finished 218001, starting 218001-219000
Finished 219001, starting 219001-220000
Finished 220001, starting 220001-221000
Finished 221001, starting 221001-222000
Finished 222001, starting 222001-223000
Finished 223001, starting 223001-224000
Finished 224001, starting 224001-225000
Finished 225001, starting 225001-226000
Finished 226001, starting 226001-227000
Finished 227001, starting 227001-228000
Finished 228001, starting 228001-229000
Finished 229001, starting 229001-230000
Finished 230001, starting 230001-231000
Finished 231001, starting 231001-232000
Finished 232001, starting 232001-233000
Finished 233001, starting 233001-234000
Finished 234001, starting 234001-235000
Finished 235001, starting 235001-236000
Finished 236001, starting 236001-237000
Finished 237001, starting 237001-238000


Finished 418001, starting 418001-419000
Finished 419001, starting 419001-420000
Finished 420001, starting 420001-421000
Finished 421001, starting 421001-422000
Finished 422001, starting 422001-423000
Finished 423001, starting 423001-424000
Finished 424001, starting 424001-425000
Finished 425001, starting 425001-426000
Finished 426001, starting 426001-427000
Finished 427001, starting 427001-428000
Finished 428001, starting 428001-429000
Finished 429001, starting 429001-430000
Finished 430001, starting 430001-431000
Finished 431001, starting 431001-432000
Finished 432001, starting 432001-433000
Finished 433001, starting 433001-434000
Finished 434001, starting 434001-435000
Finished 435001, starting 435001-436000
Finished 436001, starting 436001-437000
Finished 437001, starting 437001-438000
Finished 438001, starting 438001-439000
Finished 439001, starting 439001-440000
Finished 440001, starting 440001-441000
Finished 441001, starting 441001-442000
Finished 442001, starting 442001-443000


Finished 623001, starting 623001-624000
Finished 624001, starting 624001-625000
Finished 625001, starting 625001-626000
Finished 626001, starting 626001-627000
Finished 627001, starting 627001-628000
Finished 628001, starting 628001-629000
Finished 629001, starting 629001-630000
Finished 630001, starting 630001-631000
Finished 631001, starting 631001-632000
Finished 632001, starting 632001-633000
Finished 633001, starting 633001-634000
Finished 634001, starting 634001-635000
Finished 635001, starting 635001-636000
Finished 636001, starting 636001-637000
Finished 637001, starting 637001-638000
Finished 638001, starting 638001-639000
Finished 639001, starting 639001-640000
Finished 640001, starting 640001-641000
Finished 641001, starting 641001-642000
Finished 642001, starting 642001-643000
Finished 643001, starting 643001-644000
Finished 644001, starting 644001-645000
Finished 645001, starting 645001-646000
Finished 646001, starting 646001-647000
Finished 647001, starting 647001-648000


Finished 828001, starting 828001-829000
Finished 829001, starting 829001-830000
Finished 830001, starting 830001-831000
Finished 831001, starting 831001-832000
Finished 832001, starting 832001-833000
Finished 833001, starting 833001-834000
Finished 834001, starting 834001-835000
Finished 835001, starting 835001-836000
Finished 836001, starting 836001-837000
Finished 837001, starting 837001-838000
Finished 838001, starting 838001-839000
Finished 839001, starting 839001-840000
Finished 840001, starting 840001-841000
Finished 841001, starting 841001-842000
Finished 842001, starting 842001-843000
Finished 843001, starting 843001-844000
Finished 844001, starting 844001-845000
Finished 845001, starting 845001-846000
Finished 846001, starting 846001-847000
Finished 847001, starting 847001-848000
Finished 848001, starting 848001-849000
Finished 849001, starting 849001-850000
Finished 850001, starting 850001-851000
Finished 851001, starting 851001-852000
Finished 852001, starting 852001-853000


...

Finished 47061001, starting 47061001-47062000
Finished 47062001, starting 47062001-47063000
Finished 47063001, starting 47063001-47064000
Finished 47064001, starting 47064001-47065000
Finished 47065001, starting 47065001-47066000
Finished 47066001, starting 47066001-47067000
Finished 47067001, starting 47067001-47068000
Finished 47068001, starting 47068001-47069000
Finished 47069001, starting 47069001-47070000
Finished 47070001, starting 47070001-47071000
Finished 47071001, starting 47071001-47072000
Finished 47072001, starting 47072001-47073000
Finished 47073001, starting 47073001-47074000
Finished 47074001, starting 47074001-47075000
Finished 47075001, starting 47075001-47076000
Finished 47076001, starting 47076001-47077000
Finished 47077001, starting 47077001-47078000
Finished 47078001, starting 47078001-47079000
Finished 47079001, starting 47079001-47080000
Finished 47080001, starting 47080001-47081000
Finished 47081001, starting 47081001-47082000
Finished 47082001, starting 470820

Finished 47240001, starting 47240001-47241000
Finished 47241001, starting 47241001-47242000
Finished 47242001, starting 47242001-47243000
Finished 47243001, starting 47243001-47244000
Finished 47244001, starting 47244001-47245000
Finished 47245001, starting 47245001-47246000
Finished 47246001, starting 47246001-47247000
Finished 47247001, starting 47247001-47248000
Finished 47248001, starting 47248001-47249000
Finished 47249001, starting 47249001-47250000
Finished 47250001, starting 47250001-47251000
Finished 47251001, starting 47251001-47252000
Finished 47252001, starting 47252001-47253000
Finished 47253001, starting 47253001-47254000
Finished 47254001, starting 47254001-47255000
Finished 47255001, starting 47255001-47256000
Finished 47256001, starting 47256001-47257000
Finished 47257001, starting 47257001-47258000
Finished 47258001, starting 47258001-47259000
Finished 47259001, starting 47259001-47260000
Finished 47260001, starting 47260001-47261000
Finished 47261001, starting 472610

Finished 47419001, starting 47419001-47420000
Finished 47420001, starting 47420001-47421000
Finished 47421001, starting 47421001-47422000
Finished 47422001, starting 47422001-47423000
Finished 47423001, starting 47423001-47424000
Finished 47424001, starting 47424001-47425000
Finished 47425001, starting 47425001-47426000
Finished 47426001, starting 47426001-47427000
Finished 47427001, starting 47427001-47428000
Finished 47428001, starting 47428001-47429000
Finished 47429001, starting 47429001-47430000
Finished 47430001, starting 47430001-47431000
Finished 47431001, starting 47431001-47432000
Finished 47432001, starting 47432001-47433000
Finished 47433001, starting 47433001-47434000
Finished 47434001, starting 47434001-47435000
Finished 47435001, starting 47435001-47436000
Finished 47436001, starting 47436001-47437000
Finished 47437001, starting 47437001-47438000
Finished 47438001, starting 47438001-47439000
Finished 47439001, starting 47439001-47440000
Finished 47440001, starting 474400

Finished 47598001, starting 47598001-47599000
Finished 47599001, starting 47599001-47600000
Finished 47600001, starting 47600001-47601000
Finished 47601001, starting 47601001-47602000
Finished 47602001, starting 47602001-47603000
Finished 47603001, starting 47603001-47604000
Finished 47604001, starting 47604001-47605000
Finished 47605001, starting 47605001-47606000
Finished 47606001, starting 47606001-47607000
Finished 47607001, starting 47607001-47608000
Finished 47608001, starting 47608001-47609000
Finished 47609001, starting 47609001-47610000
Finished 47610001, starting 47610001-47611000
Finished 47611001, starting 47611001-47612000
Finished 47612001, starting 47612001-47613000
Finished 47613001, starting 47613001-47614000
Finished 47614001, starting 47614001-47615000
Finished 47615001, starting 47615001-47616000
Finished 47616001, starting 47616001-47617000
Finished 47617001, starting 47617001-47618000
Finished 47618001, starting 47618001-47619000
Finished 47619001, starting 476190

Finished 47777001, starting 47777001-47778000
Finished 47778001, starting 47778001-47779000
Finished 47779001, starting 47779001-47780000
Finished 47780001, starting 47780001-47781000
Finished 47781001, starting 47781001-47782000
Finished 47782001, starting 47782001-47783000
Finished 47783001, starting 47783001-47784000
Finished 47784001, starting 47784001-47785000
Finished 47785001, starting 47785001-47786000
Finished 47786001, starting 47786001-47787000
Finished 47787001, starting 47787001-47788000
Finished 47788001, starting 47788001-47789000
Finished 47789001, starting 47789001-47790000
Finished 47790001, starting 47790001-47791000
Finished 47791001, starting 47791001-47792000
Finished 47792001, starting 47792001-47793000
Finished 47793001, starting 47793001-47794000
Finished 47794001, starting 47794001-47795000
Finished 47795001, starting 47795001-47796000
Finished 47796001, starting 47796001-47797000
Finished 47797001, starting 47797001-47798000
Finished 47798001, starting 477980

In [7]:
from tqdm import tqdm
pd.concat([pd.DataFrame(x) for x in tqdm(allresults)]).to_csv('E:/Aaron/random_lls/all_PILR_corrs.csv')

100%|██████████| 47751954/47751954 [1:21:15<00:00, 9795.07it/s]  


In [2]:
##### load blebbistatin data

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'paranitroblebbistatin/'
if not os.path.exists(savedir):
    os.makedirs(savedir)
pilr_fl = datadir + 'PILRs/'
pilragg_fl = savedir + 'avgPILRs/'
if not os.path.exists(pilragg_fl):
    os.makedirs(pilragg_fl)
    
binframe = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
binframe = binframe[['cell']+[x for x in binframe.columns.to_list() if 'bin' in x]]

TotalFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)
TotalFrame = TotalFrame.drop(columns = [x for x in TotalFrame.columns.to_list() if 'bins' in x])
TotalFrame = TotalFrame.merge(binframe, right_on = 'cell', left_on = 'cell')

centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)

nbins = np.max(TotalFrame[[x for x in TotalFrame.columns.to_list() if 'bin' in x]].to_numpy())

#restrict dataframe
df = TotalFrame[TotalFrame.Treatment.isin(['DMSO','Para-Nitro-Blebbistatin'])].copy()

In [3]:
########### linearize blebbistatin data and animate them

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'paranitroblebbistatin/'
if not os.path.exists(savedir):
    os.makedirs(savedir)


origin = [5, 7]
whichpcs = [2,7]
binrange = 10
direction = 'counterclockwise'

coeff_df = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)

#### linearize cycle in CGPS
angframe = linear_cycle_utils.linearize_cycle_continuous(
            df,
            centers,
            origin,
            whichpcs,  
            binrange, 
            direction,)


# coeff_df = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
# coeffframe = angframe.merge(coeff_df[[x for x in coeff_df.columns.to_list() if 'shco' in x]+['cell']],
#                               left_on = 'cell', right_on = 'cell')
# for t, treat in coeffframe.groupby('Treatment'):
#     linear_cycle_utils.animate_linear_cycle_shcoeffs(
#                             coeffframe,
#                             savedir,
#                             t,
#                             whichpcs,
#                             binrange,
#                             lmax = 10,
#                             smooth = False
#                             )

#### get the average PILRs around the cycle
if __name__ ==  '__main__':
    linear_cycle_utils.get_linear_cycle_PILRs(
                        angframe, #pandas dataframe with the CGPS angular coordinates
                        savedir,
                        pilr_fl,
                        )


#### combine the PILRs around the cycle into one movie
structurelist=[]
for treat in angframe.Treatment.unique():
    for x in os.listdir(savedir + f'PC{whichpcs[0]}-PC{whichpcs[1]}_Cycle_AllSHCoeff_Visualization/{treat}/avgPILRs/'):
        struct = x.split('_')[-2]
        if struct not in structurelist and 'aggmorph' in x: structurelist.append(struct)
    for structure in structurelist:
        linear_cycle_utils.combine_linear_PILRs(savedir,
                                                structure,
                                                whichpcs,
                                                )

Finished frame_0_DMSO_actin
Finished frame_10_DMSO_actin
Finished frame_20_DMSO_actin
Finished frame_30_DMSO_actin
Finished frame_40_DMSO_actin
Finished frame_50_DMSO_actin
Finished frame_60_DMSO_actin
Finished frame_70_DMSO_actin
Finished frame_80_DMSO_actin
Finished frame_90_DMSO_actin
Finished frame_100_DMSO_actin
Finished frame_110_DMSO_actin
Finished frame_120_DMSO_actin
Finished frame_130_DMSO_actin
Finished frame_140_DMSO_actin
Finished frame_150_DMSO_actin
Finished frame_160_DMSO_actin
Finished frame_170_DMSO_actin
Finished frame_180_DMSO_actin
Finished frame_190_DMSO_actin
Finished frame_200_DMSO_actin
Finished frame_210_DMSO_actin
Finished frame_220_DMSO_actin
Finished frame_230_DMSO_actin
Finished frame_240_DMSO_actin
Finished frame_250_DMSO_actin
Finished frame_260_DMSO_actin
Finished frame_270_DMSO_actin
Finished frame_280_DMSO_actin
Finished frame_290_DMSO_actin
Finished frame_300_DMSO_actin
Finished frame_310_DMSO_actin
Finished frame_320_DMSO_actin
Finished frame_330_DM

Finished frame_270_Para-Nitro-Blebbistatin_nucleus
Finished frame_280_Para-Nitro-Blebbistatin_nucleus
Finished frame_290_Para-Nitro-Blebbistatin_nucleus
Finished frame_300_Para-Nitro-Blebbistatin_nucleus
Finished frame_310_Para-Nitro-Blebbistatin_nucleus
Finished frame_320_Para-Nitro-Blebbistatin_nucleus
Finished frame_330_Para-Nitro-Blebbistatin_nucleus
Finished frame_340_Para-Nitro-Blebbistatin_nucleus
Finished frame_350_Para-Nitro-Blebbistatin_nucleus


In [4]:
##### load ck666 data

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'ck666/'
if not os.path.exists(savedir):
    os.makedirs(savedir)
pilr_fl = datadir + 'PILRs/'
pilragg_fl = savedir + 'avgPILRs/'
if not os.path.exists(pilragg_fl):
    os.makedirs(pilragg_fl)
    
binframe = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
binframe = binframe[['cell']+[x for x in binframe.columns.to_list() if 'bin' in x]]

TotalFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)
TotalFrame = TotalFrame.drop(columns = [x for x in TotalFrame.columns.to_list() if 'bins' in x])
TotalFrame = TotalFrame.merge(binframe, right_on = 'cell', left_on = 'cell')

centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)

nbins = np.max(TotalFrame[[x for x in TotalFrame.columns.to_list() if 'bin' in x]].to_numpy())

#restrict dataframe
df = TotalFrame[TotalFrame.Treatment.isin(['DMSO','CK666'])].copy()

In [5]:
########### linearize blebbistatin data and animate them

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'ck666/'
if not os.path.exists(savedir):
    os.makedirs(savedir)


origin = [5, 7]
whichpcs = [2,7]
binrange = 10
direction = 'counterclockwise'

coeff_df = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)

#### linearize cycle in CGPS
angframe = linear_cycle_utils.linearize_cycle_continuous(
            df,
            centers,
            origin,
            whichpcs,  
            binrange, 
            direction,)


coeff_df = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
coeffframe = angframe.merge(coeff_df[[x for x in coeff_df.columns.to_list() if 'shco' in x]+['cell']],
                              left_on = 'cell', right_on = 'cell')
for t, treat in coeffframe.groupby('Treatment'):
    linear_cycle_utils.animate_linear_cycle_shcoeffs(
                            coeffframe,
                            savedir,
                            t,
                            whichpcs,
                            binrange,
                            lmax = 10,
                            smooth = False
                            )

#### get the average PILRs around the cycle
if __name__ ==  '__main__':
    linear_cycle_utils.get_linear_cycle_PILRs(
                        angframe, #pandas dataframe with the CGPS angular coordinates
                        savedir,
                        pilr_fl,
                        )


#### combine the PILRs around the cycle into one movie
structurelist=[]
for treat in angframe.Treatment.unique():
    for x in os.listdir(savedir + f'PC{whichpcs[0]}-PC{whichpcs[1]}_Cycle_AllSHCoeff_Visualization/{treat}/avgPILRs/'):
        struct = x.split('_')[-2]
        if struct not in structurelist and 'aggmorph' in x: structurelist.append(struct)
    for structure in structurelist:
        linear_cycle_utils.combine_linear_PILRs(savedir,
                                                structure,
                                                whichpcs,
                                                )

Finished frame_0_CK666_actin
Finished frame_10_CK666_actin
Finished frame_20_CK666_actin
Finished frame_30_CK666_actin
Finished frame_40_CK666_actin
Finished frame_50_CK666_actin
Finished frame_60_CK666_actin
Finished frame_70_CK666_actin
Finished frame_80_CK666_actin
Finished frame_90_CK666_actin
Finished frame_100_CK666_actin
Finished frame_110_CK666_actin
Finished frame_120_CK666_actin
Finished frame_130_CK666_actin
Finished frame_140_CK666_actin
Finished frame_150_CK666_actin
Finished frame_160_CK666_actin
Finished frame_170_CK666_actin
Finished frame_180_CK666_actin
Finished frame_190_CK666_actin
Finished frame_200_CK666_actin
Finished frame_210_CK666_actin
Finished frame_220_CK666_actin
Finished frame_230_CK666_actin
Finished frame_240_CK666_actin
Finished frame_250_CK666_actin
Finished frame_260_CK666_actin
Finished frame_270_CK666_actin
Finished frame_280_CK666_actin
Finished frame_290_CK666_actin
Finished frame_300_CK666_actin
Finished frame_310_CK666_actin
Finished frame_320_

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:/Aaron/random_lls/ck666//PC2-PC7_Cycle_AllSHCoeff_Visualization/avgPILRs/'